<a href="https://colab.research.google.com/github/wiesehahn/waldmaske/blob/master/notebooks/data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/wiesehahn/waldmaske/blob/master/notebooks/data_visualization.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/wiesehahn/waldmaske/blob/master/notebooks/data_visualization.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). 

In [0]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()    

# Visualize Data

## LUCAS Survey
https://land.copernicus.eu/imagery-in-situ/lucas

The LUCAS survey, carried out by EUROSTAT on a three-yearly basis since 2006, focuses on the state and the dynamics of changes in land use and cover in the European Union. The survey is carried out in-situ; a large number of observations are made and registered throughout the EU. Since 2012, all 27 EU countries have been covered and over 270,000 points have been analysed on different land cover types (cropland, grassland, forest, built-up areas, transport network, etc.). On these points, the surveyors have examined the land cover and land use, irrigation management and structural elements in the landscape.

(used as reference data)

In [0]:
# create Map
Map = emap.Map(center=[52,9], zoom=9)

# Add WMS Layer (LUCAS data)
lucas_url = 'https://image.discomap.eea.europa.eu/arcgis/services/LUCAS/LUCAS_2018/MapServer/WMSServer?'
Map.add_wms_layer(url=lucas_url, layers='1', name='LUCAS 2018', format='image/png', shown=False)

# Add x y z tiles 
# google_url = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'
# Map.add_tile_layer(google_url, name='Google Map', attribution='Google')

Map.addLayerControl()
Map

## EU-DEM
https://land.copernicus.eu/imagery-in-situ/eu-dem

EU-DEM v1.0 is a digital surface model (DSM) of EEA39 countries representing the first surface as illuminated by the sensors. It is a hybrid product based on SRTM and ASTER GDEM data fused by a weighted averaging approach. The statistical validation of EU-DEM v1.0 documents a relatively unbiased (-0.56 meters) overall vertical accuracy of 2.9 meters RMSE, which is fully within the contractual specification of 7m RMSE (European Commission 2009). Please read the full report here.

The following corrections and improvements have been implemented in EU-DEM v1.1:
* systematic correction of geo-positioning issues (found and corrected for Malta and Lampedusa islands);
* bias adjustment with ICESat;
* screening and removal of artefacts, including the presence of blunders (i.e. negative or positive anomalies); more than 75.000 artefacts have been detected and corrected;
* consistency with the upgraded version of EU-Hydro, in order to produce a better river network topology.


## High Resolution Image Mosaics
https://land.copernicus.eu/imagery-in-situ/european-image-mosaics/high-resolution

High Resolution satellite imagery forms the input for the creation of many information products and services, such as land cover maps or high resolution layers on land cover characteristics. Having all the satellite imagery available to cover 39 countries of EEA (EEA39), it is worthwhile to process the individual image scenes into a seamless pan-European ortho-rectified mosaic.

The HR Mosaic for 2018 has been produced using Sentinel 2 data in 10 meter resolution over the EEA 39. This data was produced at a Sentinel 2 tile level and consisted of 1079 Sentinel 2 tiles. 

In [0]:
# create Map
Map = emap.Map(center=[52,9], zoom=9)


# Add Earth Engine dataset (SRTM)
srtm = ee.Image('USGS/SRTMGL1_003')

vis_srtm = {
  'min': 0,
  'max': 1000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

Map.addLayer(srtm, vis_srtm, 'SRTM DEM', False)

dem_url = 'https://image.discomap.eea.europa.eu/arcgis/services/Elevation/EUElev_DEM_V11/MapServer/WmsServer?'
Map.add_wms_layer(url=dem_url, layers='DEM_v11_Masked2', name='EU-DEM v1.1', format='image/png', shown=False)

# hrim_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLand/HRIM_HR_FalseColour_2018/ImageServer/WMSServer'
# Map.add_wms_layer(url=hrim_url, layers='0', name='High Resolution Image Mosaic 2018 - False Colour (10m)', format='image/jpeg', shown=False)

Map.addLayerControl()
Map

## Copernicus High Resolution Layers

### Forests
https://land.copernicus.eu/pan-european/high-resolution-layers/forests

The HRL Forests consists of 3 types of (status) products, and additional change products. The status products are available for 2012 and 2015 reference years:

* Tree cover density (TCD) (level of tree cover density in a range from 0-100%)
*Dominant leaf type (DLT) (broadleaved or coniferous majority)
*A Forest type product (FTY). The forest type product allows to get as close as possible to the FAO forest definition. In its original (20m) resolution it consists of two products: 1) a dominant leaf type product that has a MMU of 0.5 ha, as well as a 10% tree cover density threshold applied, and 2) a support layer that maps, based on the dominant leaf type product, trees under agricultural use and in urban context (derived from CLC and imperviousness 2009 data). For the final 100m product trees under agricultural use and urban context from the support layer are removed.

Please find a more detailed product specification in the [technical document](https://land.copernicus.eu/pan-european/high-resolution-layers/forests/resolveuid/7a979c884e1649e882b71ba2f99b2c80).

### Grassland
https://land.copernicus.eu/pan-european/high-resolution-layers/grassland

The main product is a binary grassland/non-grassland product in 20m pixels size, that is supposed to include the full spectrum of grassland use intensity (from natural to managed grasslands). Please find a more detailed product specification in the [technical document](https://land.copernicus.eu/pan-european/high-resolution-layers/grassland/resolveuid/8f075cec57bd4fe1907829937c751703).

### Small Woody Features
https://land.copernicus.eu/pan-european/high-resolution-layers/small-woody-features

The HRL Small Woody Features (SWF) is a new CLMS product, which provides harmonized information on linear structures such as hedgerows, as well as patches (200 m² ≤ area ≤ 5000 m²) of woody features across the EEA39 countries. Please find a more detailed product specification in the [technical document](https://land.copernicus.eu/pan-european/high-resolution-layers/small-woody-features/resolveuid/94fd0d3e19ed449eba2f2ad0560cd09b).

## Related Pan-European Products

### European Settlement map
https://land.copernicus.eu/pan-european/GHSL/european-settlement-map

The European Settlement Map is a spatial raster dataset that is mapping human settlements in Europe based on SPOT5 and SPOT6 satellite imagery.

## Local Data

### Urban Atlas
https://land.copernicus.eu/local/urban-atlas

The Urban Atlas provides pan-European comparable land cover and land use data for Functional Urban Areas (FUA). The full dataset, when completed, will cover 788 FUAs covering EU27 + EFTA countries + West Balkans + Turkey + UK. It is currently composed of 438 Functional Urban Areas (FUA).

The products are:
* The 2018 edition of the Land Cover/Land Use product of Urban Atlas. 
* The 2012-2018 change product, and
* The revised 2012 edition of Urban Atlas
* Street Tree Layer (STL)

The nomenclature of the Land Cover/Land Use product is the same as for the 2012 version. It includes 17 urban classes with MMU 0.25 ha and 10 Rural Classes with MMU 1ha and can be found [here](https://land.copernicus.eu/user-corner/technical-library/urban_atlas_2012_2018_mapping_guide_v6-1.pdf).




In [0]:
# create Map
Map = emap.Map(center=[52,9], zoom=9)

# imp_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLandPublic/HRL_ImperviousnessDensity_2015/MapServer/WmsServer?'
# Map.add_wms_layer(url=stl_url, layers=['0'], name='Copernicus HRL - Imperviousness 2015', format='image/png', shown=True)

tcd_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLandPublic/HRL_TreeCoverDensity_2015/MapServer/WmsServer?'
Map.add_wms_layer(url=tcd_url, layers=['0', '1'], name='Copernicus HRL - Tree Cover Density 2015', format='image/png', shown=False)

dlt_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLandPublic/HRL_DominantLeafType_2015/MapServer/WmsServer?'
Map.add_wms_layer(url=dlt_url, layers=['0'], name='Copernicus HRL - Dominant Leaf Type 2015', format='image/png', shown=False)

fty_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLandPublic/HRL_ForestType_2015/MapServer/WmsServer?'
Map.add_wms_layer(url=fty_url, layers=['HRL Forest Type 2015 20m', 'HRL Forest Type 2015 100m'], name='Copernicus HRL - Forest Type 2015', format='image/png', shown=False)

fadsl_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLandPublic/HRL_ForestAdditionalSupportLayer_2015/MapServer/WmsServer?'
Map.add_wms_layer(url=fadsl_url, layers=['0'], name='Copernicus HRL - Forest Additional Support Layer 2015', format='image/png', shown=False)

grass_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLandPublic/HRL_Grassland_2015/MapServer/WmsServer?'
Map.add_wms_layer(url=grass_url, layers=['0', '1'], name='Copernicus HRL - Grassland 2015', format='image/png', shown=False)

swf_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLandPublic/HRL_SmallWoodyFeatures_2015_005m/ImageServer/WMSServer'
Map.add_wms_layer(url=swf_url, layers=['0'], name='Copernicus HRL - Small Woody Features 2015', format='image/png', shown=False)

esm_url = 'https://image.discomap.eea.europa.eu/arcgis/services/GioLandPublic/ESM2015_R2019_WM/ImageServer/WMSServer'
Map.add_wms_layer(url=esm_url, layers=['0'], name='European Settlement Map 2015', format='image/png', shown=False)

ua_url = 'https://image.discomap.eea.europa.eu/arcgis/services/UrbanAtlas/UA_UrbanAtlas_2018/MapServer/WmsServer?'
Map.add_wms_layer(url=ua_url, layers=['Land Use vector', 'Land Use Raster'], name='Copernicus UA - Urban Atlas 2018', format='image/png', shown=False)

stl_url = 'https://image.discomap.eea.europa.eu/arcgis/services/UrbanAtlas/UA_StreetTreeLayer_2018/MapServer/WmsServer?'
Map.add_wms_layer(url=stl_url, layers=['STL 2018 Vector', 'STL 2018 Raster'], name='Copernicus UA - Street Tree Layer 2018', format='image/png', shown=False)


Map.addLayerControl()
Map